In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import KFold

In [5]:
random_seed = 42

# Zbiór "mały"
auth_data_path = "data/everything-4/authentic"
spoof_data_path = "data/everything-4/spoof"

# Zbiór "duży"
#auth_data_path = "data/everything-1/authentic"
#spoof_data_path = "data/everything-1/spoof"

# Zbiór "duży" z internetu
#auth_data_path = "data/everything-2/authentic"
#spoof_data_path = "data/everything-2/spoof"

# Load authentic images
authentic_images = []
authentic_image_names = []

files = [f for f in os.listdir(auth_data_path) if f.endswith(".jpg") or f.endswith("jpeg") or f.endswith(".png") or f.endswith(".webp")]

for file in files:
    image = cv2.imread(os.path.join(auth_data_path, file))
    authentic_images.append(image)
    authentic_image_names.append(file)
    
# Load spoof images
spooof_images = []
spoof_image_names = []

files = [f for f in os.listdir(spoof_data_path) if f.endswith(".jpg") or f.endswith("jpeg") or f.endswith(".png") or f.endswith(".webp")]

for file in files:
    image = cv2.imread(os.path.join(spoof_data_path, file))
    spooof_images.append(image)
    spoof_image_names.append(file)
    
# Create X and y from all images
X = authentic_images + spooof_images
names = authentic_image_names + spoof_image_names
y = np.concatenate((np.zeros(len(authentic_images)), np.ones(len(spooof_images))))

# KFold
kf = KFold(n_splits=5, random_state=random_seed, shuffle=True)

# Save splits to /folds/fold0 etc..
for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train = [X[i] for i in train_idx]
    y_train = y[train_idx]
    X_test = [X[i] for i in test_idx]
    y_test = y[test_idx]
    
    # Save to disk
    for idx, img in enumerate(X_train):
        name = names[train_idx[idx]].split('\\')[-1].split('.')[0]
    if y_train[idx] == 0:
        cv2.imwrite(f'data/folds/fold{fold}/train/authentic/{name}.png', img)
    else:
        cv2.imwrite(f'data/folds/fold{fold}/train/spoof/{name}.png', img)

    for idx, img in enumerate(X_test):
        name = names[test_idx[idx]].split('\\')[-1].split('.')[0]
        if y_test[idx] == 0:
            cv2.imwrite(f'data/folds/fold{fold}/test/authentic/{name}.png', img)
        else:
            cv2.imwrite(f'data/folds/fold{fold}/test/spoof/{name}.png', img)